### Links
* https://community.plotly.com/t/announcing-plotly-py-4-8-plotly-express-support-for-wide-and-mixed-form-data-plus-a-pandas-backend/40048
* https://plotly.com/python/styling-plotly-express/
* https://plotly.com/python/line-charts/
* https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e#:~:text=We're%20excited%20to%20announce%20the%20release%20of%20JupyterDash%2C%20our,and%20R%20and%20Julia!)
* https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import pandas as pd
import numpy as np

from darts import TimeSeries
from darts.models import  AutoARIMA
                          

import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [ ]:
df = pd.read_csv("../../data/later/profile_growth.csv")
df['Date'] = pd.to_datetime(df['Date'])
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, 'Date', 'Followers')

In [ ]:
df['Followers'].shape[0]

In [ ]:
# Create figures in Express

# 90 days change
days_increase = str(round((df['Followers'][0] - df['Followers'][90]) / df['Followers'][90] * 100,2)) + "%"

figure1 = go.Figure()
figure1.add_trace(go.Scatter(x=df['Date'], y=df['Followers'],
                    mode='lines',
                    name='Followers',
                    line=dict(color='rgb(115,115,115)', width=2),
                    connectgaps=True)
                    )

figure1.add_annotation(
        text=days_increase,
             xref="x domain", 
             yref="y domain",
             # The arrow head will be 25% along the x axis, starting from the left
             x=0.25, 
            # The arrow head will be 40% along the y axis, starting from the bottom
             y=0.40, 
             font=dict(
            family="Courier New, monospace",
            size=18,
            color="#ffffff"
            ),
            showarrow=False,
            bordercolor='rgb(67,67,67)',
            borderwidth=1,
            borderpad=10,
            bgcolor='rgb(189,189,189)',
            opacity=0.8
        )

figure1.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        showspikes=True
    ),
    autosize=True,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    font_family="Courier New",
    title_font_family="Times New Roman",
    title="Growth",
    hovermode="x unified",
    legend_title_text=str(days_increase) + "%",
    legend = dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=1,
    font=dict(
            family="Arial",
            size=12,
            color="black"
    ),
    ),
    plot_bgcolor='white'
)


In [ ]:
title = 'Impressions & Reach'
labels = ['Impressions', 'Reach']


figure2 = go.Figure()

figure2.add_trace(go.Scatter(x=df['Date'], y=df['Impressions'],
                    mode='lines+markers',
                    name='Impressions',
                    line=dict(color='rgb(67,67,67)', width=2),
                    connectgaps=True))
figure2.add_trace(go.Scatter(x=df['Date'], y=df['Reach'],
                    mode='lines+markers',
                    name='Reach',
                    line=dict(color='rgb(189,189,189)', width=2),
                    connectgaps=True))


figure2.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        showspikes=True
    ),
    autosize=True,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    font_family="Courier New",
    title_font_family="Times New Roman",
    hovermode="x unified",
    legend_title_text='Discovery',
    legend = dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    font=dict(
            family="Arial",
            size=12,
            color="black"
    ),
    ),
    plot_bgcolor='white'
)

In [ ]:
model = AutoARIMA()
model.fit(series)
forecast = model.predict(7)
series_df = series.pd_dataframe().reset_index()#.set_index('component')
series_df.to_csv("../../resources/series_df.csv", index = False)
forecast_df = forecast.pd_dataframe().reset_index()
forecast_df.to_csv("../../resources/forecast_df.csv", index = False)

In [ ]:
# Create traces
figure3 = go.Figure()
figure3.add_trace(go.Scatter(x=series_df['Date'], y=series_df['Followers'],
                    mode='lines',
                    name='ToDate',
                    line=dict(color='rgb(67,67,67)', width=2),
                    connectgaps=True))
figure3.add_trace(go.Scatter(x=forecast_df['Date'], y=forecast_df['Followers'],
                    mode='lines+markers',
                    name='Forecast',
                    line=dict(color='rgb(49,130,189)', width=2),
                    connectgaps=True))

figure3.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        showspikes=True
    ),
    autosize=True,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    font_family="Courier New",
    title_font_family="Times New Roman",
    hovermode="x unified",
    legend_title_text='Forecast',
    legend = dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    font=dict(
            family="Arial",
            size=12,
            color="black"
    ),
    ),
    plot_bgcolor='white'
)

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("JupyterDash Demo"),
     html.Div([dcc.Graph(figure=figure1)]),
     html.Div([dcc.Graph(figure=figure2)]),
     html.Div([dcc.Graph(figure=figure3)])
     
    ])

# Run app and display result inline in the notebook
app.run_server(mode='external')